The special method names are always written with leading and trailing double underscores (i.e., \_\_getitem\_\_). For example, the syntax obj[key] is supported by the \_\_getitem\_\_ special method. In order to evaluate my_collection[key], the interpreter calls my_collection\.\__getitem\__(key).

![](images/Capture1.JPG)

#### A Pythonic Card Deck
The following is a very simple example, but it demonstrates the power of implementing just two special methods, \__getitem\__ and \__len\__.

A class to represent a deck of playing cards.

In [20]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
        #list comprehension
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        #list comprehension
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

namedtuple can be used to build classes of objects that are just bundles of attributes with no custom methods, like a database record.

Example:

In [18]:
beer_card = Card('7', 'diamonds')

beer_card

Card(rank='7', suit='diamonds')

But the point of this example is the FrenchDeck class. It’s short, but it packs a punch.

First, like any standard Python collection, a deck responds to the len() function by returning the number of cards in it:

In [23]:
deck = FrenchDeck()

len(deck)

52

Reading specific cards from the deck—say, the first or the last—should be as easy as deck[0] or deck[-1], and this is what the \__getitem\__ method provides:

In [24]:
deck[0]

Card(rank='2', suit='spades')

In [25]:
deck[-1]

Card(rank='A', suit='hearts')

We’ve just seen two advantages of using special methods to leverage the Python data model:

- The users of your classes don’t have to memorize arbitrary method names for standard operations (“How to get the number of items? Is it .size(), .length(), or what?”).
- It’s easier to benefit from the rich Python standard library and avoid reinventing the wheel, like the random.choice function.

Look at the top three cards from a brand new deck:

In [27]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

Pick just the aces by starting on index 12 and skipping 13 cards at a time:

In [28]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

Just by implementing the \__getitem\__ special method, our deck is also iterable:

In [31]:
for card in deck:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')
Card(rank='Q', suit='spades')
Card(rank='K', suit='spades')
Card(rank='A', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='diamonds')
Card(rank='5', suit='diamonds')
Card(rank='6', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='8', sui

in works with our FrenchDeck class because it is iterable. Check it out:

In [32]:
Card('Q', 'hearts') in deck

True

How about sorting? 
- Here is a function that ranks cards by that rule, returning 0 for the 2 of clubs and 51 for the ace of spades:

In [33]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [34]:
for card in sorted(deck, key=spades_high):
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

- Although FrenchDeck implicitly inherits from object,its functionality is not inherited, but comes from leveraging the data model and composition. 
- By implementing the special methods \__len\__ and \__getitem\__, our FrenchDeck behaves like a standard Python sequence, allowing it to benefit from core language features (e.g., iteration and slicing) and from the standard library, as shown by the examples using random.choice, reversed, and sorted. 
- Thanks to composition, the \__len\__ and \__getitem\__ implementations can hand off all the work to a list object, self._cards.

![](images\Capture2.JPG)

#### How Special Methods Are Used

- The first thing to know about special methods is that they are meant to be called by the Python interpreter, and not by you. 
- You don’t write my_object.\__len\__(). You write len(my_object) and, if my_object is an instance of a user-defined class, then Python calls the \__len\__ instance method you implemented.

- More often than not, the special method call is implicit. For example, the statement for i in x: actually causes the invocation of iter(x), which in turn may call x.\__iter\__() if that is available.

- If you need to invoke a special method, it is usually better to call the related built-in function (e.g., len, iter, str, etc). 
- These built-ins call the corresponding special method, but often provide other services and—for built-in types—are faster than method calls. 

#### EMULATING NUMERIC TYPES

We will implement a class to represent two-dimensional vectors—that is Euclidean vectors like those used in math and physics

In [37]:
from math import hypot

class Vector:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [38]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)

v1 + v2

Vector(4, 5)

The abs built-in function returns the absolute value of integers and floats, and the magnitude of complex numbers, so to be consistent, our API also uses abs to calculate the magnitude of a vector:

In [39]:
abs(v1)

4.47213595499958

In [41]:
#scalar multiplication
v1 * 3

Vector(6, 12)

#### String Representation

The \__repr\__ special method is called by the repr built-in to get the string representation of the object for inspection. If we did not implement \__repr\__, vector instances would be shown in the console like < Vector object at 0x10e100070 >.

#### ARITHMETIC OPERATORS

Implements two operators: + and *, to show basic usage of \__add\__ and \__mul\__. Note that in both cases, the methods create and return a new instance of Vector, and do not modify either operand—self or other are merely read.

#### BOOLEAN VALUE OF A CUSTOM TYPE

In [43]:
bool(v1)

True

#### Overview of Special Methods

The Python Language Reference lists 83 special method names, 47 of which are used to implement arithmetic, bitwise, and comparison operators.

![](images\Capture4.JPG)

![](images\Capture4.JPG)

![](images\Capture5.JPG)

#### Summary:

- By implementing special methods, your objects can behave like the built-in types, enabling the expressive coding style the community considers Pythonic.

- A basic requirement for a Python object is to provide usable string representations of itself, one used for debugging and logging, another for presentation to end users. That is why the special methods \__repr\__ and \__str\__ exist in the data model.

- Emulating sequences, as shown with the FrenchDeck example, is one of the most widely used applications of the special methods.